In [ ]:
from google.colab import files   #train데이터 업로드
uploaded = files.upload()

In [ ]:
from google.colab import files  #test데이터 업로드
uploaded = files.upload()

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 데이터 불러오기 (학습 데이터, 테스트 데이터)
데이터 분석 단계(4.2_농구선수_데이터분석.ipynb)에서 생성한 농구 선수 포지션 예측하기의  
학습 데이터 및 테스트 데이터를 로드합니다.

In [ ]:
with open('basketball_train.pkl', 'rb') as train_data:
    train = pickle.load(train_data)                     #train 변수에 80명 샘플 
    
with open('basketball_test.pkl', 'rb') as test_data:
    test = pickle.load(test_data)                       #test 변수에 20명 샘플

In [26]:
train.head()

,Player,Pos,3P,TRB,BLK
54,Frank Kaminsky,C,1.5,4.5,0.5
16,Willie Cauley-Stein,C,0.0,4.5,0.6
12,Devin Booker,SG,1.9,3.2,0.3
40,Montrezl Harrell,C,0.0,3.8,0.7
0,Alex Abrines,SG,1.4,1.3,0.1


# 최적의 k 찾기 (교차 검증 - cross validation)

In [ ]:
# import kNN library
from sklearn.neighbors import KNeighborsClassifier   #knn
from sklearn.model_selection import cross_val_score  #교차검증

# find best k, range from 3 to half of the number of data   #k=3,5,7....39[40]
max_k_range = train.shape[0] // 2   #80//2 = 40  몫나눗셈
k_list = []
for i in range(3, max_k_range, 2):  #(3,40,2),  i=3,5,7,9,....39
    k_list.append(i)              #k_lst=[1,3,5,...39] k값 후보 목록

cross_validation_scores = []   #k값에 따른 정확도 저장 리스트
x_train = train[['3P', 'BLK' , 'TRB']]   #학습에 사용할 특징
y_train = train[['Pos']]                #학습에 사용할 레이블

# 10-fold cross validation
for k in k_list:   # [3,5,7,,,,39]
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train.values.ravel(),
                             cv=10, scoring='accuracy')
    cross_validation_scores.append(scores.mean())

print(cross_validation_scores)

In [ ]:
# visualize accuracy according to k
plt.plot(k_list, cross_validation_scores)
plt.xlabel('the number of k')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# find best k
cvs = cross_validation_scores
k = k_list[cvs.index(max(cross_validation_scores))]
print("The best number of k : " + str(k) )

# 2개의 특징으로 예측하기 (3점슛, 블로킹) -> 테스트

In [ ]:
# import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=11)

# select data features
x_train = train[['3P', 'BLK']]  #특징
# select target value
y_train = train[['Pos']]        #레이블

# setup knn using train data
knn.fit(x_train, y_train.values.ravel())   #fit , 주어진 학습데이터를 이용해서 분류기를 생성

# select data feature to be used for prediction
x_test = test[['3P', 'BLK']]

# select target value
y_test = test[['Pos']]

# test
pred = knn.predict(x_test)   #predict 예측 함수, 테스트 데이터의 특징을 가지고 레이블 예측히여 변수에 저장

In [ ]:
# check ground_truth with knn prediction   #그라운드투르쓰: 테스트데이터의 실제 레이블값
comparison = pd.DataFrame(
    {'prediction':pred, 'ground_truth':y_test.values.ravel()}) 
comparison   #변수이름

In [ ]:
# check accuracy
print("accuracy : "+ 
          str(accuracy_score(y_test.values.ravel(), pred)) )   #그라운드트루쓰와 예측값을 주고 정확도 수치 계산

# 3개의 특징으로 예측하기 (3점슛, 블로킹, 리바운드)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)

# select data features to be used in train
x_train = train[['3P', 'BLK', 'TRB']]
# select target
y_train = train[['Pos']]

# build knn model
knn.fit(x_train, y_train.values.ravel())

# select features to be used for prediction
x_test = test[['3P', 'BLK', 'TRB']]

# select target
y_test = test[['Pos']]

# test
pred = knn.predict(x_test)

In [ ]:
# check ground_truth with knn prediction
comparison = pd.DataFrame(
    {'prediction':pred, 'ground_truth':y_test.values.ravel()}) 
comparison

In [ ]:
# check accuracy
print("accuracy : " + 
          str( accuracy_score(y_test.values.ravel(), pred)) )